# __WSI - ćwiczenie 5.__

### __Sztuczne sieci neuronowe__

#### __Treść ćwiczenia__

- Celem cwiczenia jest implementacja perceptronu wielowarstwowego oraz wybranego algorytmu
optymalizacji gradientowej z algorytmem propagacji wstecznej.
- Nastepnie nalezy wytrenowac perceptron wielowarstwowy do klasyfikacji zbioru danych wine
(https://archive.ics.uci.edu/ml/datasets/wine). Zbiór ten dostepny jest w pakiecie scikitlearn
(sklearn.datasets.load wine).

In [132]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
# confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, auc, RocCurveDisplay, PrecisionRecallDisplay, recall_score, precision_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from seaborn import heatmap
import plotly.express as px
from math import log, inf, e, tanh, sqrt
from sklearn.utils import resample, shuffle
import unittest
from sklearn.datasets import load_wine
from scipy.special import expit, logit, softmax


RNG = np.random.default_rng()

cele: 
- perceptron wielowarstwowy, implementacja ze zmienną ilością warstw głębokich oraz zmienną ilością długości wektora neuronów
- kilka algorytmów optymalizacji wag sieci (gradient prosty, SGD, algorytm ewolucyjny??)

zadania:
1. model sieci
2. propagacja wsteczna
3. optymalizacja wag

__Weight matrix per layer:__

$$
\theta^{l}=
\left[\begin{array}{ccc}
\omega_{1,1}& \cdots&\omega_{1,k+1}\\
\vdots&\ddots&\vdots\\
\omega_{n,1}&\cdots&\omega_{n,k+1}
\end{array}\right]
$$
where  $ \omega_{i,j} $ is the $j$-th weight of the $i$-th neuron (in layer $l$), and $ \omega_{i,k+1} $ is its bias

__Matrix of layers:__

$$
\Theta=
\left[\begin{array}{ccc}
\theta^{1}& \cdots&\theta^{\lambda}\
\end{array}\right]
$$
where  $ \theta^{\lambda} $ is the output layer 

__Input data vector:__

$$
y^0=\left[\begin{array}{ccc}
x^T& 1
\end{array}\right]^T
$$
it is extended by 1 to allow easier multiplication

__Opertation of a single neuron:__

$$
y^l_i=\psi(\theta^l_i y^{l-1})
$$

$\psi$ is the neuron activation function

__Output layer:__

$$
f_i(x, \Theta)=\theta^\lambda_i y^\lambda
$$

__Backpropagation__
$$
\frac{de}{ds^l_i}=\frac{de}{dy^l_i}\frac{∂\psi^l(s^l_i)}{∂s^l_i}
$$

$$
\frac{de}{dy^l_i}=\sum_{\gamma} \frac{de}{ds^{l+1}_\gamma}\theta^{l+1}_{\gamma,i} 
$$
For the last layer this can be calculated immidiately:

$$
\frac{∂e}{∂\theta^l_{i,j}}=\frac{de}{ds^l_i}y^{l-1}_j= \frac{de}{dy^l_i} \frac{∂\psi^l(s^l_i)}{∂s^l_i}y^{l-1}_j
$$

For the rest:
$$
\frac{∂e}{∂\theta^l_{i,j}}= \left( \sum_{\gamma} \frac{de}{ds^{l+1}_\gamma}\theta^{l+1}_{\gamma,i} \right) \frac{∂\psi^l(s^l_i)}{∂s^l_i}y^{l-1}_j
$$

So in order to calculate all the derivatives we need to first calculate all derivatives of neuron input sums

__Default weigths initialization:__

todo

In [133]:
class MLP():
    """
    todo fully-connected?

    Attribubtes:
        _layers: 

    Methods:
        fit:
    """
    def __init__(self, dimensions:list, feature_number:int, activations:list=None, loss_func:str = None) -> None:
        """
        todo

        Args:
            dimensions: starting from first hidden layer
            activations: last actvation function should be linear if a basic MLP is being modeled

        Returns:
            MLP object

        Raises:
            None
        """

        # todo assertions

        self._layers = np.array([np.empty((dimensions[0], feature_number + 1))] + \
                                [np.empty((dimensions[i+1], dimensions[i]+1)) for i in range(len(dimensions)-1)], 
                                dtype=object)
        
        self._activation_names = activations
        self._activations = []
        self._derivatives = []
        
        activation_funcs = {
            'linear': lambda x: x,
            'logistic':lambda x: expit(x),
            'tanh': lambda x: tanh(x),
            'ReLU':lambda x: max(0, x),
            'softmax': lambda x: softmax(x)
        }
        activation_derivs = {
            'linear': lambda x: 1,
            'logistic':lambda x: expit(x) * (1 - expit(x)),
            'tanh': lambda x: tanh(x)**2,
            'ReLU':lambda x: 1*(x>0),
            'softmax':...
        }

        if activations:
            self._activations = [activation_funcs[name] for name in activations]
            self._derivatives = [activation_derivs[name] for name in activations]

        loss_funcs = {
            'MSE': mean_squared_error,
            'crossEntropy': log_loss
        }

        loss_func_derivs = {
            'MSE': lambda v, t: 2*(v - t),
            'crossEntropy': lambda v, t: -(t*log(v) + (1-t)*log(1-v))
            }

        self._loss_func = None
        self._loss_func_deriv = None

    def initialize_weights(self, strategy='default'):
        # todo strategies
        if strategy == 'default':
            for layer in self._layers[:-1]:
                size = layer.shape[1]
                size_sqrt = sqrt(size)
                with np.nditer(layer, op_flags=['writeonly']) as it:
                    for w in it:
                        w[...] = RNG.uniform(-1/size_sqrt, 1/size_sqrt)
            self._layers[-1].fill(0)
        if strategy == 'ones':
            for layer in self._layers:
                layer.fill(1)

    def fit(self):
        pass

    def feed_forward(self, input_vector):
        # todo description
        # this returns a vector consisting of tuples 
        # of single neuron input sum and effective activated output 
        all_inputs_outputs = []
        current_layer_activated_outputs = input_vector
        for layer, activate in zip(self._layers, self._activations):
            current_layer_activated_outputs = current_layer_activated_outputs + [1]
            current_layer_input_sums = [np.matmul(weights, current_layer_activated_outputs) for weights in layer]
            current_layer_activated_outputs = [activate(s) for s in current_layer_input_sums]
            current_layer_pairs = np.array([current_layer_input_sums, current_layer_activated_outputs]).T
            all_inputs_outputs.append(current_layer_pairs)
        return all_inputs_outputs

    def propagate(self, input_vector):
        return self.feed_forward(input_vector)[-1][:, 1].tolist()

    def backprop(self, input_vector, true_output_vector, loss_func_derivative):
        
        gradient_estimate = []
        
        all_inputs_outputs = self.feed_forward(input_vector)
        all_partial_deriv_input_sums = [[deriv(s) for s in layer[:, 0]] for layer, deriv in zip(all_inputs_outputs, self._derivatives)]
        
        # configure for output layer
        # calculate loss derivative values
        total_deriv_outputs = [loss_func_derivative(output, true_value) for output, true_value in zip(all_inputs_outputs[-1][:,1], true_output_vector)]
        # all outputs appended with 1 (bias) at each layer
        all_outputs_with_input_vector = [input_vector + [1]] + [layer[:,1].tolist() + [1] for layer in all_inputs_outputs]
        # for each layer
        layer_idx = -1
        for layer_idx in range(len(self._layers)-1, -1, -1):
            partial_deriv_input_sums = all_partial_deriv_input_sums[layer_idx]
            total_deriv_input_sums = np.multiply(total_deriv_outputs, partial_deriv_input_sums)
            # calculate estimated gradient for layer 
            layer_gradient = np.matmul(total_deriv_input_sums[np.newaxis].T, np.array(all_outputs_with_input_vector[layer_idx])[np.newaxis])
            gradient_estimate.insert(0, layer_gradient)
            
            # prepare for next layer 
            if not layer_idx == 0:
                # shorten the arrays by 1 to avoid calculating derivatives for biases
                total_deriv_outputs = [np.multiply(total_deriv_input_sums, neuron_weights).sum() for neuron_weights in self._layers[layer_idx][:,:-1].T]

        return np.array(gradient_estimate, dtype=object)

    def BGD_step(self, data, target_label, learning_rate):
        gradient_sum = np.shape(self._layers)
        for _, row in data.iterrows():
            gradient_sum = gradient_sum + self.backprop(row.drop([target_label]).tolist(), 
                                                         [row[target_label]], 
                                                         lambda v, t: 2*(v-t))
        avg_gradient = gradient_sum / data.shape[0]
        self._layers = self._layers - (avg_gradient * learning_rate)
    
    def SGD_step(self, data, mini_batch_size):
        ...

    def train(self, train_data, target_label, learning_rate, strategy='BGD', epoch_number=100, mini_batch_size=0.1, monitor=False, test_data=None):
        errors = []
        for _ in range(epoch_number):
            if strategy == 'BGD':
                self.BGD_step(train_data, target_label, learning_rate)
            if strategy == 'SGD':
                ...
            if monitor:
                pred = self.predict(test_data.drop(target_label, axis=1)).apply(lambda row: row[0])
                mse = mean_squared_error(test_data[target_label], pred)
                errors.append(mse)
        if monitor:
            return errors

            

    def predict(self, data):
        return data.apply(lambda row : self.propagate(row.tolist()), axis=1)


In [134]:
class TestMLP(unittest.TestCase):

    def test_init_1_neuron(self):
        mlp = MLP([1], 1)
        self.assertEqual(len(mlp._layers), 1)
        self.assertEqual(mlp._layers[0].shape, (1, 2))

    def test_init_multi_neuron(self):
        mlp = MLP([5, 3, 11], 15)
        self.assertEqual(len(mlp._layers), 3)
        self.assertEqual(mlp._layers[0].shape, (5, 16))
        self.assertEqual(mlp._layers[1].shape, (3, 6))
        self.assertEqual(mlp._layers[2].shape, (11, 4))

    def test_1_feature_feed_forward_1_neuron(self):
        mlp = MLP([1], 1)
        mlp._activations = [lambda x: x]
        mlp._layers[0] = np.array([[3, 2]])
        self.assertEqual(mlp.propagate([3]), [11.])

    def test_multi_features_feed_forward_mutli_neuron(self):
        mlp = MLP([2, 3, 2], 2)
        mlp._activations = [lambda x: x, lambda x: 2*x, lambda x: x]
        mlp.initialize_weights('ones')
        self.assertEqual(mlp.propagate([2,  3]), [79., 79.])

    def test_initialize_weigths_default(self):
        mlp = MLP([2, 3, 2], 2)
        mlp._activations = [lambda x: x, lambda x: 2*x, lambda x: x]
        mlp.initialize_weights(strategy='default')
        for layer in mlp._layers[:-1]:
            self.assertTrue(((layer > -1) & (layer < 1)).all())
        self.assertTrue((mlp._layers[-1] == 0).all())

    def test_single_predict(self):
        mlp = MLP([2, 3, 2], 2)
        mlp._activations =[lambda x: x, lambda x: 2*x, lambda x: x]
        mlp._derivatives = [lambda x: 1, lambda x: 2, lambda x: 1]
        mlp.initialize_weights('ones')
        self.assertEqual(mlp.propagate([2,  3]), [79, 79])

    def test_backprop(self):
        mlp = MLP([2, 3, 1], 1)
        mlp._activations = [lambda x: x**2, lambda x: x**2, lambda x: x]
        mlp._derivatives = [lambda x: 2*x, lambda x: 2*x, lambda x: 1]
        mlp.initialize_weights('ones') 
        gradient = mlp.backprop([3], [3000], lambda v, t: 2*(v - t))
        true_gradient = np.array([np.array([[2547072.,  849024.],
                                   [2547072.,  849024.]]),
                         np.array([[566016., 566016.,  35376.],
                                   [566016., 566016.,  35376.],
                                   [566016., 566016.,  35376.]]),
                         np.array([[5.83704e+05, 5.83704e+05, 5.83704e+05, 5.36000e+02]])], dtype=object)
        self.assertTrue(all([(ar1 == ar2).all() for ar1, ar2 in zip (gradient, true_gradient)]))

unittest.main(argv=[''],  exit=False)

.......
----------------------------------------------------------------------
Ran 7 tests in 0.006s

OK


In [135]:
func = lambda x: x**2
data = pd.DataFrame(data=[[x, func(x)] for x in RNG.uniform(-1, 1, 100)], columns=['x', 'y'])

In [136]:
x_train, x_test, y_train, y_test = train_test_split(data['x'], data['y'], test_size=0.2, random_state=32)
train = x_train.to_frame().join(y_train.to_frame())
test = x_test.to_frame().join(y_test.to_frame())

In [137]:
mlp = MLP([5, 1], 1, ['tanh', 'linear'])
mlp.initialize_weights('default')
mlp.train(train, 'y', 0.2, strategy='BGD', epoch_number=1000)

In [138]:
test['pred'] = mlp.predict(x_test.to_frame()).apply(lambda x: x[0])
px.line(test.sort_values('x'), x='x', y=['y',  'pred'])

In [139]:
features, target = load_wine(return_X_y=True, as_frame=True)
data = features.join(target)
data

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2
